In [1]:

import os

In [2]:

%pwd

'c:\\Users\\Hp\\Text-Summarizer-Project\\research'

In [3]:

os.chdir("../")

In [4]:

%pwd

'c:\\Users\\Hp\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [9]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

[2024-06-17 17:46:42,873: INFO: config: PyTorch version 2.2.2 available.]


In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-06-17 18:07:21,823: INFO: common: yaml file: config\config.yaml loaded successfully]


[2024-06-17 18:07:21,837: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-17 18:07:21,840: INFO: common: created directory at: artifacts]
[2024-06-17 18:07:21,846: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/920 [00:00<?, ?it/s]

{'loss': 3.1995, 'grad_norm': 20.48751449584961, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 3.2262, 'grad_norm': 9.711017608642578, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}
{'loss': 2.9224, 'grad_norm': 11.304835319519043, 'learning_rate': 3e-06, 'epoch': 0.03}
{'loss': 2.853, 'grad_norm': 13.299054145812988, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.04}
{'loss': 2.6911, 'grad_norm': 11.871971130371094, 'learning_rate': 5e-06, 'epoch': 0.05}
{'loss': 2.729, 'grad_norm': 23.833356857299805, 'learning_rate': 6e-06, 'epoch': 0.07}
{'loss': 2.6316, 'grad_norm': 10.161120414733887, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.08}
{'loss': 2.4347, 'grad_norm': 9.823800086975098, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.09}
{'loss': 2.5044, 'grad_norm': 7.497169494628906, 'learning_rate': 9e-06, 'epoch': 0.1}
{'loss': 2.467, 'grad_norm': 6.793668270111084, 'learning_rate': 1e-05, 'epoch': 0.11}
{'loss': 2.2325, 'grad_norm': 8.96264

  0%|          | 0/818 [00:00<?, ?it/s]

{'eval_loss': 1.4850271940231323, 'eval_runtime': 1263.5113, 'eval_samples_per_second': 0.647, 'eval_steps_per_second': 0.647, 'epoch': 0.54}
{'loss': 1.7053, 'grad_norm': 4.268325328826904, 'learning_rate': 4.880952380952381e-05, 'epoch': 0.55}
{'loss': 1.6668, 'grad_norm': 3.2583422660827637, 'learning_rate': 4.761904761904762e-05, 'epoch': 0.56}
{'loss': 1.6925, 'grad_norm': 3.8473622798919678, 'learning_rate': 4.642857142857143e-05, 'epoch': 0.58}
{'loss': 1.5671, 'grad_norm': 5.758591175079346, 'learning_rate': 4.523809523809524e-05, 'epoch': 0.59}
{'loss': 1.6613, 'grad_norm': 3.9351396560668945, 'learning_rate': 4.404761904761905e-05, 'epoch': 0.6}
{'loss': 1.6979, 'grad_norm': 5.0593581199646, 'learning_rate': 4.2857142857142856e-05, 'epoch': 0.61}
{'loss': 1.7027, 'grad_norm': 7.10544490814209, 'learning_rate': 4.166666666666667e-05, 'epoch': 0.62}
{'loss': 1.6329, 'grad_norm': 3.5524168014526367, 'learning_rate': 4.047619047619048e-05, 'epoch': 0.63}
{'loss': 1.5351, 'grad_no

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8, 'forced_eos_token_id': 1}


{'loss': 1.6163, 'grad_norm': 3.763716697692871, 'learning_rate': 0.0, 'epoch': 1.0}
{'train_runtime': 119827.716, 'train_samples_per_second': 0.123, 'train_steps_per_second': 0.008, 'train_loss': 1.8227204094762388, 'epoch': 1.0}
